# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))
    all_train_edges = adj_train.nonzero()
    out_train_edges = [[all_train_edges[0][i],all_train_edges[1][i]] for i in range(all_train_edges[0].shape[0])]
    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return np.array(out_train_edges), val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    train_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0], replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [6]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)

# 定义模型

In [7]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [8]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 4.8
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Wisconsin'

In [9]:
all_best_auc = []
all_best_ap = []
for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)
    
"""
train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_context_embeddings = context_embedding.weight.detach()

importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_importance_embeddings = importance_embedding.weight.detach()
    
auc_first_best_auc = 0
auc_first_best_ap = 0
ap_first_best_auc = 0
ap_first_best_ap = 0

for module_epoch in range(100):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
    update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
    nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
"""

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 252.8948
Epoch: 01, Loss: 252.4199
Epoch: 02, Loss: 251.5387
Epoch: 03, Loss: 250.3283
Epoch: 04, Loss: 248.8668
AP SCORE：  0.5211134300106531
AUC SCORE:  0.4831320283215327
Epoch: 00, Loss: 247.2269
Epoch: 01, Loss: 247.1487
Epoch: 02, Loss: 247.0024
Epoch: 03, Loss: 246.7982
Epoch: 04, Loss: 246.5465
AP SCORE：  0.5280581888739736
AUC SCORE:  0.497292794668888
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 246.2568
Epoch: 01, Loss: 245.9233
Epoch: 02, Loss: 245.2971
Epoch: 03, Loss: 244.4203
Epoch: 04, Loss: 243.3351
AP SCORE：  0.5403066736135129
AUC SCORE:  0.5212411495210328
Epoch: 00, Loss: 242.0810
Epoch: 01, Loss: 242.0303
Epoch: 02, Loss: 241.9348
Epoch: 03, Loss: 241.7999
Epoch: 04, Loss: 241.6310
AP SCORE：  0.5446777009810054
AUC SCORE:  0.529362765514369
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 241.4334
Epoch: 01, Loss: 241.1535


Epoch: 00, Loss: 187.4514
Epoch: 01, Loss: 187.4037
Epoch: 02, Loss: 187.3134
Epoch: 03, Loss: 187.1853
Epoch: 04, Loss: 187.0241
AP SCORE：  0.7151601453827636
AUC SCORE:  0.7117867555185339
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 186.8340
Epoch: 01, Loss: 186.6844
Epoch: 02, Loss: 186.4011
Epoch: 03, Loss: 185.9987
Epoch: 04, Loss: 185.4913
AP SCORE：  0.7177555991967906
AUC SCORE:  0.7138692211578508
Epoch: 00, Loss: 184.8917
Epoch: 01, Loss: 184.8439
Epoch: 02, Loss: 184.7534
Epoch: 03, Loss: 184.6252
Epoch: 04, Loss: 184.4641
AP SCORE：  0.7182732407821764
AUC SCORE:  0.7134527280299875
######################### 模块循环 ： 21 ##########################
Epoch: 00, Loss: 184.2745
Epoch: 01, Loss: 184.1281
Epoch: 02, Loss: 183.8506
Epoch: 03, Loss: 183.4567
Epoch: 04, Loss: 182.9598
AP SCORE：  0.7191991650426919
AUC SCORE:  0.7155351936693045
Epoch: 00, Loss: 182.3726
Epoch: 01, Loss: 182.3251
Epoch: 02, Loss: 182.2353
Epoch: 03, Loss: 182.1083
Epoch:

Epoch: 04, Loss: 146.7766
AP SCORE：  0.7485706403729727
AUC SCORE:  0.7480216576426488
Epoch: 00, Loss: 146.3865
Epoch: 01, Loss: 146.3671
Epoch: 02, Loss: 146.3307
Epoch: 03, Loss: 146.2793
Epoch: 04, Loss: 146.2151
AP SCORE：  0.7482415067709699
AUC SCORE:  0.7471886713869221
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 146.1403
Epoch: 01, Loss: 146.0459
Epoch: 02, Loss: 145.8671
Epoch: 03, Loss: 145.6133
Epoch: 04, Loss: 145.2934
AP SCORE：  0.7493055162141511
AUC SCORE:  0.7480216576426488
Epoch: 00, Loss: 144.9158
Epoch: 01, Loss: 144.8975
Epoch: 02, Loss: 144.8631
Epoch: 03, Loss: 144.8145
Epoch: 04, Loss: 144.7540
AP SCORE：  0.750746687226348
AUC SCORE:  0.7496876301541024
######################### 模块循环 ： 41 ##########################
Epoch: 00, Loss: 144.6832
Epoch: 01, Loss: 144.5920
Epoch: 02, Loss: 144.4190
Epoch: 03, Loss: 144.1737
Epoch: 04, Loss: 143.8644
AP SCORE：  0.7509867000052727
AUC SCORE:  0.749271137026239
Epoch: 00, Loss: 143.4993

Epoch: 04, Loss: 125.4787
AP SCORE：  0.7425932918761533
AUC SCORE:  0.7480216576426489
Epoch: 00, Loss: 125.2745
Epoch: 01, Loss: 125.2670
Epoch: 02, Loss: 125.2529
Epoch: 03, Loss: 125.2329
Epoch: 04, Loss: 125.2078
AP SCORE：  0.7414239316305309
AUC SCORE:  0.7476051645147855
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 125.1783
Epoch: 01, Loss: 125.1288
Epoch: 02, Loss: 125.0350
Epoch: 03, Loss: 124.9017
Epoch: 04, Loss: 124.7333
AP SCORE：  0.7410575476071783
AUC SCORE:  0.7476051645147855
Epoch: 00, Loss: 124.5339
Epoch: 01, Loss: 124.5266
Epoch: 02, Loss: 124.5130
Epoch: 03, Loss: 124.4936
Epoch: 04, Loss: 124.4693
AP SCORE：  0.7417659781655488
AUC SCORE:  0.7488546438983757
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 124.4407
Epoch: 01, Loss: 124.3924
Epoch: 02, Loss: 124.3008
Epoch: 03, Loss: 124.1705
Epoch: 04, Loss: 124.0060
AP SCORE：  0.741355192137703
AUC SCORE:  0.749271137026239
Epoch: 00, Loss: 123.8112

Epoch: 04, Loss: 113.1872
AP SCORE：  0.7355173058602404
AUC SCORE:  0.75551853394419
Epoch: 00, Loss: 113.0475
Epoch: 01, Loss: 113.0428
Epoch: 02, Loss: 113.0339
Epoch: 03, Loss: 113.0213
Epoch: 04, Loss: 113.0055
AP SCORE：  0.7360841992842767
AUC SCORE:  0.7559350270720534
######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 112.9868
Epoch: 01, Loss: 112.9528
Epoch: 02, Loss: 112.8884
Epoch: 03, Loss: 112.7966
Epoch: 04, Loss: 112.6805
AP SCORE：  0.736736430733035
AUC SCORE:  0.75676801332778
Epoch: 00, Loss: 112.5427
Epoch: 01, Loss: 112.5381
Epoch: 02, Loss: 112.5294
Epoch: 03, Loss: 112.5171
Epoch: 04, Loss: 112.5015
AP SCORE：  0.7367640395756684
AUC SCORE:  0.7567680133277801
######################### 模块循环 ： 81 ##########################
Epoch: 00, Loss: 112.4832
Epoch: 01, Loss: 112.4497
Epoch: 02, Loss: 112.3861
Epoch: 03, Loss: 112.2957
Epoch: 04, Loss: 112.1812
AP SCORE：  0.7375492662537523
AUC SCORE:  0.7580174927113702
Epoch: 00, Loss: 112.0454
Ep

...

Epoch: 03, Loss: 149.3508
Epoch: 04, Loss: 149.0780
AP SCORE：  0.7332498061124291
AUC SCORE:  0.7396917950853811
Epoch: 00, Loss: 148.7550
Epoch: 01, Loss: 148.7384
Epoch: 02, Loss: 148.7070
Epoch: 03, Loss: 148.6624
Epoch: 04, Loss: 148.6061
AP SCORE：  0.7329558961289471
AUC SCORE:  0.7388588088296543
######################### 模块循环 ： 39 ##########################
Epoch: 00, Loss: 148.5398
Epoch: 01, Loss: 148.4612
Epoch: 02, Loss: 148.3120
Epoch: 03, Loss: 148.0999
Epoch: 04, Loss: 147.8320
AP SCORE：  0.7327205367078594
AUC SCORE:  0.7392753019575177
Epoch: 00, Loss: 147.5146
Epoch: 01, Loss: 147.4980
Epoch: 02, Loss: 147.4664
Epoch: 03, Loss: 147.4217
Epoch: 04, Loss: 147.3652
AP SCORE：  0.7328610404136447
AUC SCORE:  0.7396917950853811
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 147.2984
Epoch: 01, Loss: 147.2211
Epoch: 02, Loss: 147.0744
Epoch: 03, Loss: 146.8657
Epoch: 04, Loss: 146.6021
AP SCORE：  0.7339080210780068
AUC SCORE:  0.74094127446897

Epoch: 03, Loss: 129.0782
Epoch: 04, Loss: 128.8964
AP SCORE：  0.7337891984903325
AUC SCORE:  0.7476051645147855
Epoch: 00, Loss: 128.6811
Epoch: 01, Loss: 128.6709
Epoch: 02, Loss: 128.6517
Epoch: 03, Loss: 128.6244
Epoch: 04, Loss: 128.5901
AP SCORE：  0.732933846168287
AUC SCORE:  0.7467721782590587
######################### 模块循环 ： 59 ##########################
Epoch: 00, Loss: 128.5496
Epoch: 01, Loss: 128.4975
Epoch: 02, Loss: 128.3985
Epoch: 03, Loss: 128.2579
Epoch: 04, Loss: 128.0800
AP SCORE：  0.7340531549789553
AUC SCORE:  0.7476051645147855
Epoch: 00, Loss: 127.8694
Epoch: 01, Loss: 127.8596
Epoch: 02, Loss: 127.8411
Epoch: 03, Loss: 127.8150
Epoch: 04, Loss: 127.7822
AP SCORE：  0.7335484369627167
AUC SCORE:  0.7471886713869221
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 127.7435
Epoch: 01, Loss: 127.6925
Epoch: 02, Loss: 127.5958
Epoch: 03, Loss: 127.4582
Epoch: 04, Loss: 127.2843
AP SCORE：  0.7341881280654973
AUC SCORE:  0.748438150770512

Epoch: 04, Loss: 115.2198
AP SCORE：  0.7390124937869242
AUC SCORE:  0.7530195751770096
Epoch: 00, Loss: 115.0723
Epoch: 01, Loss: 115.0661
Epoch: 02, Loss: 115.0544
Epoch: 03, Loss: 115.0379
Epoch: 04, Loss: 115.0171
AP SCORE：  0.7390124937869242
AUC SCORE:  0.7530195751770096
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 114.9928
Epoch: 01, Loss: 114.9571
Epoch: 02, Loss: 114.8895
Epoch: 03, Loss: 114.7935
Epoch: 04, Loss: 114.6721
AP SCORE：  0.7384738193739341
AUC SCORE:  0.7526030820491462
Epoch: 00, Loss: 114.5284
Epoch: 01, Loss: 114.5225
Epoch: 02, Loss: 114.5115
Epoch: 03, Loss: 114.4959
Epoch: 04, Loss: 114.4763
AP SCORE：  0.7379385985379614
AUC SCORE:  0.7517700957934195
######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 114.4533
Epoch: 01, Loss: 114.4185
Epoch: 02, Loss: 114.3526
Epoch: 03, Loss: 114.2589
Epoch: 04, Loss: 114.1405
AP SCORE：  0.738058426689002
AUC SCORE:  0.7517700957934194
Epoch: 00, Loss: 114.000

Epoch: 01, Loss: 106.1413
Epoch: 02, Loss: 106.1320
Epoch: 03, Loss: 106.1188
Epoch: 04, Loss: 106.1020
AP SCORE：  0.7363752638295191
AUC SCORE:  0.7505206164098294
######################### 模块循环 ： 99 ##########################
Epoch: 00, Loss: 106.0820
Epoch: 01, Loss: 106.0570
Epoch: 02, Loss: 106.0093
Epoch: 03, Loss: 105.9415
Epoch: 04, Loss: 105.8557
AP SCORE：  0.7361195926646994
AUC SCORE:  0.750104123281966
Epoch: 00, Loss: 105.7540
Epoch: 01, Loss: 105.7488
Epoch: 02, Loss: 105.7390
Epoch: 03, Loss: 105.7251
Epoch: 04, Loss: 105.7074
AP SCORE：  0.7359886661962772
AUC SCORE:  0.7496876301541026


'\ntrain_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)\ncontext_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入\nnodes_context_embeddings = context_embedding.weight.detach()\n\nimportance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入\nnodes_importance_embeddings = importance_embedding.weight.detach()\n    \nauc_first_best_auc = 0\nauc_first_best_ap = 0\nap_first_best_auc = 0\nap_first_best_ap = 0\n\nfor module_epoch in range(100):\n    print("######################### 模块循环 ： %d ##########################"%module_epoch)\n    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)\n    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)\n    if auc > auc_first_best_auc:\n        auc_first_best_auc = auc\n        auc_first_best_ap = ap\n    if ap > ap_first_bes

In [10]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())

"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.7739676855117102
STD AP :  0.030092936479349835
MEAN AUC :  0.7705955851728448
STD AUC :  0.021400452848379277


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'